Link to Kaggle dataset: https://www.kaggle.com/datasets/deathtrooper/multichannel-glaucoma-benchmark-dataset/data

In [1]:
import zipfile
import pandas as pd
import keras
import os
import numpy as np
from keras.layers import Dense, Activation, Flatten, Dropout, GlobalAveragePooling2D, Conv2D, BatchNormalization, MaxPooling2D, Input, Concatenate, ReLU, AveragePooling2D, UpSampling2D, GlobalMaxPooling2D
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import SGD, Adam
from keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.applications.mobilenet_v3 import preprocess_input 
from keras.layers import GlobalMaxPool2D

In [2]:
# Unzip files for processing
with zipfile.ZipFile("full-fundus.zip","r") as zip_ref:
    zip_ref.extractall("full-fundus")
    
with zipfile.ZipFile("blood-vessel.zip","r") as zip_ref:
    zip_ref.extractall("blood-vessel")


In [3]:
# Read in metadata, and filter for a specific subset of images
# TODO: will add more variety of images later
image_data = pd.read_csv('metadata.csv')
image_data = image_data[image_data['names'].str.contains('FIVES', case=True) | image_data['names'].str.contains('HAGIS', case=True)
                       | image_data['names'].str.contains('LES-AV', case=True) | image_data['names'].str.contains('G1020', case=True)
                       | image_data['names'].str.contains('OIA', case=True)]
#g1020 = image_data[image_data['names'].str.contains('G1020', case=True)]
#g1020 = g1020[g1020["fundus_oc_seg"] != "Not Visible"]

#image_data = pd.concat([subset_image_data, g1020])
image_data = image_data[['types', 'fundus', 'names']]

In [4]:
# There are some invalid data types for relevant columns
print(image_data.info())

<class 'pandas.core.frame.DataFrame'>
Index: 5946 entries, 0 to 12448
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   types   5946 non-null   int64 
 1   fundus  5946 non-null   object
 2   names   5946 non-null   object
dtypes: int64(1), object(2)
memory usage: 185.8+ KB
None


In [5]:
# Change datatypes to the desired values
image_data['types'] = image_data['types'].astype('string')
image_data['fundus'] = image_data['fundus'].astype('string')
image_data['names'] = image_data['names'].astype('string')
image_data['image_names'] = image_data['names'] + '.png'


In [6]:
print(image_data.info())

<class 'pandas.core.frame.DataFrame'>
Index: 5946 entries, 0 to 12448
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   types        5946 non-null   string
 1   fundus       5946 non-null   string
 2   names        5946 non-null   string
 3   image_names  5946 non-null   string
dtypes: string(4)
memory usage: 232.3 KB
None


In [7]:
# Randomize data 
image_data_random = image_data.sample(frac=1, random_state=1)

# Split into healthy and glaucoma positive sets
healthy = image_data_random[image_data_random['types'] == "0"]
glaucoma = image_data_random[image_data_random['types'] == "1"]

print(f"Number of healthy samples: {len(healthy)}")
print(f"Number of unhealthy samples: {len(glaucoma)}")

Number of healthy samples: 5136
Number of unhealthy samples: 758


In [8]:
# Partition data into test and train sets
healthy_train_size = 3595
glaucoma_train_size = 530

healthy_test_subset = image_data_random.head(len(healthy) - healthy_train_size)
glaucoma_test_subset = image_data_random.head(len(glaucoma) - glaucoma_train_size)
test_subset = pd.concat([healthy_test_subset, glaucoma_test_subset])

healthy_train_subset = image_data_random.tail(healthy_train_size)
glaucoma_train_subset = image_data_random.tail(glaucoma_train_size)
train_subset = pd.concat([healthy_train_subset, glaucoma_train_subset])

print(f"Test size: {len(test_subset)}")
print(f"Train size: {len(train_subset)}")

Test size: 1769
Train size: 4125


In [9]:
train_data_generator = ImageDataGenerator(preprocessing_function= preprocess_input)

flow_train_data = train_data_generator.flow_from_dataframe(dataframe=train_subset, 
                                            batch_size= 8, 
                                            shuffle=True, 
                                            x_col="image_names", 
                                            y_col="types", 
                                            validate_filenames=True, 
                                            target_size=(224, 224), 
                                            directory='full-fundus/full-fundus/', 
                                            color_mode='rgb')

test_data_generator = ImageDataGenerator(preprocessing_function= preprocess_input)

flow_test_data = train_data_generator.flow_from_dataframe(dataframe=test_subset, 
                                            batch_size= 1, 
                                            shuffle=False, 
                                            x_col="image_names", 
                                            y_col="types", 
                                            validate_filenames=True, 
                                            target_size=(224, 224), 
                                            directory='full-fundus/full-fundus/', 
                                            color_mode='rgb')



Found 4125 validated image filenames belonging to 3 classes.
Found 1769 validated image filenames belonging to 3 classes.


In [10]:
def conv_model(image_size):
    
    m_input = keras.Input(shape=(image_size, image_size, 3))
    transfer = keras.applications.MobileNetV3Large(
        weights='imagenet', include_top= False, input_tensor= m_input, alpha=0.75
    )
    m_output = Dropout(0.5)(transfer.output)
    m_output = Conv2D(filters=256, kernel_size=1)(m_output) 
    m_output = GlobalMaxPool2D()(m_output)
    m_output = Dropout(0.5)(m_output)
    m_output = Dense(3, activation='sigmoid')(m_output)

    return keras.Model(inputs=m_input, outputs=m_output)

model = conv_model(image_size=224)

# Reduce the learning rate if an epoch occurs where there is no improvement to the output of the loss function
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.8, patience=1, min_lr=1e-6)

# Using the Adam optimizer with binary cross entropy, compile the model using the given metrics
model.compile(optimizer= Adam(1e-5), 
              loss='binary_crossentropy', 
              metrics=['binary_accuracy',keras.metrics.AUC(),keras.metrics.Precision(), keras.metrics.Recall()])

trained_model = model.fit(flow_train_data, 
                    steps_per_epoch= len(flow_train_data) // 8,
                    validation_data= flow_test_data, 
                    validation_steps= len(flow_test_data), 
                    epochs=20, 
                    callbacks=[reduce_lr])

Epoch 1/20


/Users/ThomasMcBride/anaconda3/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


64/64 ━━━━━━━━━━━━━━━━━━━━ 37s 460ms/step - auc: 0.2524 - binary_accuracy: 0.3996 - loss: 6.0127 - precision: 0.1078 - recall: 0.1095 - val_auc: 0.3732 - val_binary_accuracy: 0.4839 - val_loss: 1.3170 - val_precision: 0.1418 - val_recall: 0.1085 - learning_rate: 1.0000e-05
Epoch 2/20
64/64 ━━━━━━━━━━━━━━━━━━━━ 13s 206ms/step - auc: 0.3184 - binary_accuracy: 0.4809 - loss: 4.5942 - precision: 0.1494 - recall: 0.1204 - val_auc: 0.0000e+00 - val_binary_accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - learning_rate: 1.0000e-05
Epoch 3/20


2024-03-29 22:58:35.428176: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
/Users/ThomasMcBride/anaconda3/lib/python3.11/contextlib.py:155: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


64/64 ━━━━━━━━━━━━━━━━━━━━ 29s 456ms/step - auc: 0.3740 - binary_accuracy: 0.4936 - loss: 4.0042 - precision: 0.2112 - recall: 0.1885 - val_auc: 0.7670 - val_binary_accuracy: 0.7426 - val_loss: 0.6295 - val_precision: 0.6650 - val_recall: 0.4590 - learning_rate: 1.0000e-05
Epoch 4/20
64/64 ━━━━━━━━━━━━━━━━━━━━ 14s 212ms/step - auc: 0.4375 - binary_accuracy: 0.5370 - loss: 3.4226 - precision: 0.2698 - recall: 0.2294 - val_auc: 0.0000e+00 - val_binary_accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - learning_rate: 1.0000e-05
Epoch 5/20


...

2024-03-29 23:04:09.263504: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


64/64 ━━━━━━━━━━━━━━━━━━━━ 35s 556ms/step - auc: 0.7838 - binary_accuracy: 0.7567 - loss: 1.3764 - precision: 0.6357 - recall: 0.6338 - val_auc: 0.9192 - val_binary_accuracy: 0.9143 - val_loss: 0.5423 - val_precision: 0.8716 - val_recall: 0.8711 - learning_rate: 5.1200e-06
Epoch 18/20
64/64 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - auc: 0.8114 - binary_accuracy: 0.8258 - loss: 0.7613 - precision: 0.8048 - recall: 0.6302 - val_auc: 0.0000e+00 - val_binary_accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - learning_rate: 5.1200e-06
Epoch 19/20


2024-03-29 23:04:45.496496: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-03-29 23:04:45.506269: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


64/64 ━━━━━━━━━━━━━━━━━━━━ 36s 560ms/step - auc: 0.8129 - binary_accuracy: 0.7761 - loss: 1.2450 - precision: 0.6705 - recall: 0.6504 - val_auc: 0.9196 - val_binary_accuracy: 0.9145 - val_loss: 0.5418 - val_precision: 0.8717 - val_recall: 0.8717 - learning_rate: 5.1200e-06
Epoch 20/20
64/64 ━━━━━━━━━━━━━━━━━━━━ 17s 274ms/step - auc: 0.8019 - binary_accuracy: 0.7660 - loss: 1.3211 - precision: 0.6458 - recall: 0.6597 - val_auc: 0.0000e+00 - val_binary_accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - learning_rate: 4.0960e-06


2024-03-29 23:05:38.889645: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


In [11]:
# Demonstration:
print(trained_model)
img_path = 'full-fundus/full-fundus/OIA-ODIR-TEST-ONLINE-252.png'
img = image.load_img(img_path, target_size=(224, 224))
 
img_array_representation = np.expand_dims(image.img_to_array(img), axis=0)
img_array_representation = preprocess_input(img_array_representation)  # Assuming preprocess_input is defined in your code

prediction = model.predict(img_array_representation)
print(prediction)
# Interpret the prediction
if prediction[0][0] > prediction[0][1] and prediction[0][0] > prediction[0][2]:
    print(f"Prediction: Glaucoma, with {prediction[0][0] * 100}% confidence")
elif prediction[0][1] > prediction[0][0] and prediction[0][1] > prediction[0][2]:
    print(f"Prediction: Healthy, with {prediction[0][1] * 100}% confidence")
else: 
    print(f"Prediction: Indecisive, with {prediction[0][2] * 100}% confidence")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 499ms/step
[[8.8388933e-04 9.9974877e-01 1.2317003e-03]]
Prediction: Healthy, with 99.97487664222717% confidence
